## Dependencies

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import multiprocessing as mp

In [2]:
train = pd.read_csv('../output/data_5_fold.csv')
test = pd.read_csv('../output/test.csv')

print('Number of train samples: ', train.shape[0])
print('Number of test samples: ', test.shape[0])
display(test.head())

Number of train samples:  19871
Number of test samples:  2690


,Unnamed: 0,id_code,diagnosis
0,0,10_left.jpeg,0
1,1,10_right.jpeg,0
2,2,13_left.jpeg,0
3,3,13_right.jpeg,0
4,4,17_left.jpeg,0


# Define variable

In [3]:
HEIGHT = 224
WIDTH = 224


data_2_path = '../data/dr_data_2/train/'
data_1_path = '../data/dr_data_1/train/'
test_path = '../data/dr_test/test/'
train_dest_path = '../dataset/train/'
test_dest_path =  '../dataset/test/'

# Pre-procecess images

In [4]:
def crop_image(img, tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
            
        return img

def circle_crop(img):
    img = crop_image(img)

    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = width//2
    y = height//2
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image(img)

    return img
        
def preprocess_image(image_id, base_path, save_path, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    print(base_path + image_id)
    image = cv2.imread(base_path + image_id)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = circle_crop(image)
    image = cv2.resize(image, (HEIGHT, WIDTH))
    cv2.imwrite(save_path + image_id, image)
    

def preprocess_data(df, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    df = df.reset_index()
    print("Train images")
    for i in range(df.shape[0]):
        item = df.iloc[i]
        image_id = item['id_code']
        item_data = item['data']
        if item_data == 1:
            preprocess_image(image_id, data_1_path, train_dest_path)
        if item_data == 2:
            preprocess_image(image_id, data_2_path, train_dest_path)

        
def preprocess_test(df, base_path=test_path, save_path=test_dest_path, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    df = df.reset_index()
    print("Test images")
    for i in range(df.shape[0]):
        image_id = df.iloc[i]['id_code']
        preprocess_image(image_id, base_path, save_path)


In [ ]:
preprocess_test(test)
preprocess_data(train)

Test images
../data/dr_test/test/10_left.jpeg
../data/dr_test/test/10_right.jpeg
../data/dr_test/test/13_left.jpeg
../data/dr_test/test/13_right.jpeg
../data/dr_test/test/17_left.jpeg
../data/dr_test/test/19_left.jpeg
../data/dr_test/test/19_right.jpeg
../data/dr_test/test/20_left.jpeg
../data/dr_test/test/20_right.jpeg
../data/dr_test/test/21_left.jpeg
../data/dr_test/test/21_right.jpeg
../data/dr_test/test/22_left.jpeg
../data/dr_test/test/22_right.jpeg
../data/dr_test/test/23_left.jpeg
../data/dr_test/test/23_right.jpeg
../data/dr_test/test/25_left.jpeg
../data/dr_test/test/25_right.jpeg
../data/dr_test/test/31_left.jpeg
../data/dr_test/test/31_right.jpeg
../data/dr_test/test/33_left.jpeg
../data/dr_test/test/33_right.jpeg
../data/dr_test/test/36_right.jpeg
../data/dr_test/test/40_right.jpeg
../data/dr_test/test/41_left.jpeg
../data/dr_test/test/41_right.jpeg
../data/dr_test/test/42_left.jpeg
../data/dr_test/test/42_right.jpeg
../data/dr_test/test/46_left.jpeg
../data/dr_test/test/4

../data/dr_test/test/375_right.jpeg
../data/dr_test/test/376_left.jpeg
../data/dr_test/test/376_right.jpeg
../data/dr_test/test/380_left.jpeg
../data/dr_test/test/380_right.jpeg
../data/dr_test/test/381_left.jpeg
../data/dr_test/test/381_right.jpeg
../data/dr_test/test/384_left.jpeg
../data/dr_test/test/384_right.jpeg
../data/dr_test/test/386_left.jpeg
../data/dr_test/test/386_right.jpeg
../data/dr_test/test/389_left.jpeg
../data/dr_test/test/389_right.jpeg
../data/dr_test/test/393_left.jpeg
../data/dr_test/test/395_left.jpeg
../data/dr_test/test/395_right.jpeg
../data/dr_test/test/398_left.jpeg
../data/dr_test/test/398_right.jpeg
../data/dr_test/test/399_left.jpeg
../data/dr_test/test/399_right.jpeg
../data/dr_test/test/400_left.jpeg
../data/dr_test/test/400_right.jpeg
../data/dr_test/test/404_left.jpeg
../data/dr_test/test/404_right.jpeg
../data/dr_test/test/408_left.jpeg
../data/dr_test/test/408_right.jpeg
../data/dr_test/test/410_left.jpeg
../data/dr_test/test/410_right.jpeg
../dat

../data/dr_test/test/773_left.jpeg
../data/dr_test/test/773_right.jpeg
../data/dr_test/test/780_left.jpeg
../data/dr_test/test/780_right.jpeg
../data/dr_test/test/783_left.jpeg
../data/dr_test/test/783_right.jpeg
../data/dr_test/test/784_left.jpeg
../data/dr_test/test/784_right.jpeg
../data/dr_test/test/787_left.jpeg
../data/dr_test/test/787_right.jpeg
../data/dr_test/test/791_left.jpeg
../data/dr_test/test/791_right.jpeg
../data/dr_test/test/792_left.jpeg
../data/dr_test/test/792_right.jpeg
../data/dr_test/test/793_left.jpeg
../data/dr_test/test/793_right.jpeg
../data/dr_test/test/796_left.jpeg
../data/dr_test/test/796_right.jpeg
../data/dr_test/test/799_left.jpeg
../data/dr_test/test/799_right.jpeg
../data/dr_test/test/800_left.jpeg
../data/dr_test/test/800_right.jpeg
../data/dr_test/test/802_left.jpeg
../data/dr_test/test/802_right.jpeg
../data/dr_test/test/803_left.jpeg
../data/dr_test/test/803_right.jpeg
../data/dr_test/test/805_left.jpeg
../data/dr_test/test/805_right.jpeg
../dat

../data/dr_test/test/1165_right.jpeg
../data/dr_test/test/1170_left.jpeg
../data/dr_test/test/1170_right.jpeg
../data/dr_test/test/1172_left.jpeg
../data/dr_test/test/1172_right.jpeg
../data/dr_test/test/1179_left.jpeg
../data/dr_test/test/1179_right.jpeg
../data/dr_test/test/1180_left.jpeg
../data/dr_test/test/1180_right.jpeg
../data/dr_test/test/1182_left.jpeg
../data/dr_test/test/1182_right.jpeg
../data/dr_test/test/1188_left.jpeg
../data/dr_test/test/1188_right.jpeg
../data/dr_test/test/1192_left.jpeg
../data/dr_test/test/1192_right.jpeg
../data/dr_test/test/1197_left.jpeg
../data/dr_test/test/1197_right.jpeg
../data/dr_test/test/1199_left.jpeg
../data/dr_test/test/1199_right.jpeg
../data/dr_test/test/1202_left.jpeg
../data/dr_test/test/1202_right.jpeg
../data/dr_test/test/1206_left.jpeg
../data/dr_test/test/1206_right.jpeg
../data/dr_test/test/1211_left.jpeg
../data/dr_test/test/1211_right.jpeg
../data/dr_test/test/1212_left.jpeg
../data/dr_test/test/1212_right.jpeg
../data/dr_tes

../data/dr_test/test/1588_left.jpeg
../data/dr_test/test/1588_right.jpeg
../data/dr_test/test/1589_left.jpeg
../data/dr_test/test/1589_right.jpeg
../data/dr_test/test/1595_left.jpeg
../data/dr_test/test/1595_right.jpeg
../data/dr_test/test/1596_left.jpeg
../data/dr_test/test/1596_right.jpeg
../data/dr_test/test/1600_left.jpeg
../data/dr_test/test/1600_right.jpeg
../data/dr_test/test/1608_left.jpeg
../data/dr_test/test/1608_right.jpeg
../data/dr_test/test/1609_left.jpeg
../data/dr_test/test/1609_right.jpeg
../data/dr_test/test/1610_left.jpeg
../data/dr_test/test/1610_right.jpeg
../data/dr_test/test/1611_left.jpeg
../data/dr_test/test/1611_right.jpeg
../data/dr_test/test/1624_left.jpeg
../data/dr_test/test/1624_right.jpeg
../data/dr_test/test/1630_left.jpeg
../data/dr_test/test/1630_right.jpeg
../data/dr_test/test/1634_left.jpeg
../data/dr_test/test/1634_right.jpeg
../data/dr_test/test/1638_left.jpeg
../data/dr_test/test/1638_right.jpeg
../data/dr_test/test/1641_left.jpeg
../data/dr_test

...

../data/dr_data_1/train/ad1f7445b1a8.png
../data/dr_data_1/train/ad20080452de.png
../data/dr_data_1/train/ad2f0b9d059c.png
../data/dr_data_1/train/ad312ca98202.png
../data/dr_data_1/train/ad3fc5076852.png
../data/dr_data_1/train/ad570b850a4f.png
../data/dr_data_1/train/ad6b07d5c338.png
../data/dr_data_1/train/ad93d88c87ea.png
../data/dr_data_1/train/ad944bd56bb6.png
../data/dr_data_1/train/adb56cecafaf.png
../data/dr_data_1/train/add1d681d712.png
../data/dr_data_1/train/addf66a50f42.png
../data/dr_data_1/train/ae1344610ebe.png
../data/dr_data_1/train/ae20112e7a1e.png
../data/dr_data_1/train/ae2c3f6312ef.png
../data/dr_data_1/train/ae2e888905ba.png
../data/dr_data_1/train/ae49cc60f251.png
../data/dr_data_1/train/ae57c8630249.png
../data/dr_data_1/train/ae58ccb5905e.png
../data/dr_data_1/train/ae5d31979f19.png
../data/dr_data_1/train/ae61e19fb766.png
../data/dr_data_1/train/ae8424cdb029.png
../data/dr_data_1/train/ae8472f8d310.png
../data/dr_data_1/train/ae94ce412de9.png
../data/dr_data_

../data/dr_data_1/train/ba4e62c11cc0.png
../data/dr_data_1/train/ba735b286d62.png
../data/dr_data_1/train/baaca2f7e1f0.png
../data/dr_data_1/train/bab776139279.png
../data/dr_data_1/train/bac1744955c2.png
../data/dr_data_1/train/bacfa2b8e706.png
../data/dr_data_1/train/bacfb1029f6b.png
../data/dr_data_1/train/badb5ff8d3c7.png
../data/dr_data_1/train/bb08949dd70a.png
../data/dr_data_1/train/bb11db08584a.png
../data/dr_data_1/train/bb2f89488ecd.png
../data/dr_data_1/train/bb45257258cc.png
../data/dr_data_1/train/bb5083fae98f.png
../data/dr_data_1/train/bb733062f494.png
../data/dr_data_1/train/bb752b179751.png
../data/dr_data_1/train/bb783d8e496f.png
../data/dr_data_1/train/bb7e0a2544cd.png
../data/dr_data_1/train/bb85097857fa.png
../data/dr_data_1/train/bb9a3d835a94.png
../data/dr_data_1/train/bba38f2294a3.png
../data/dr_data_1/train/bc23f74e14dd.png
../data/dr_data_1/train/bc34ed91c9bc.png
../data/dr_data_1/train/bc34f52c37c7.png
../data/dr_data_1/train/bc73ce76ec43.png
../data/dr_data_

../data/dr_data_1/train/c97472ef2c66.png
../data/dr_data_1/train/c98f623d08d1.png
../data/dr_data_1/train/c9c563864ab1.png
../data/dr_data_1/train/c9d42d7534e0.png
../data/dr_data_1/train/c9e697117f3f.png
../data/dr_data_1/train/c9ea9d5eab65.png
../data/dr_data_1/train/c9f0dc2c8b43.png
../data/dr_data_1/train/ca05f7e7801b.png
../data/dr_data_1/train/ca0f1a17c8e5.png
../data/dr_data_1/train/ca1036496659.png
../data/dr_data_1/train/ca25745942b0.png
../data/dr_data_1/train/ca2b54b95ade.png
../data/dr_data_1/train/ca30a97e9d13.png
../data/dr_data_1/train/ca360bec5851.png
../data/dr_data_1/train/ca63fe4f4b52.png
../data/dr_data_1/train/ca6842bfcbc9.png
../data/dr_data_1/train/ca7140ecf389.png
../data/dr_data_1/train/ca7570c5925c.png
../data/dr_data_1/train/ca7f5caddf96.png
../data/dr_data_1/train/ca891d37a43c.png
../data/dr_data_1/train/ca9c912ebad7.png
../data/dr_data_1/train/cab3dfa7962d.png
../data/dr_data_1/train/cac40227d3b2.png
../data/dr_data_1/train/cac80797770f.png
../data/dr_data_

../data/dr_data_1/train/d7e5fe5245e0.png
../data/dr_data_1/train/d801c0a66738.png
../data/dr_data_1/train/d803598dabda.png
../data/dr_data_1/train/d807c53c1399.png
../data/dr_data_1/train/d81338217fc5.png
../data/dr_data_1/train/d81b6ed83bc2.png
../data/dr_data_1/train/d838d5b9f571.png
../data/dr_data_1/train/d83c3efade75.png
../data/dr_data_1/train/d83d0695e215.png
../data/dr_data_1/train/d8404680bba6.png
../data/dr_data_1/train/d844a7252f4e.png
../data/dr_data_1/train/d85588ff2ebd.png
../data/dr_data_1/train/d85a842d20bd.png
../data/dr_data_1/train/d85d052900b4.png
../data/dr_data_1/train/d85ea1220a03.png
../data/dr_data_1/train/d865997a6280.png
../data/dr_data_1/train/d866c26d76f0.png
../data/dr_data_1/train/d868acdccb5b.png
../data/dr_data_1/train/d871895742b1.png
../data/dr_data_1/train/d881c04f01fe.png
../data/dr_data_1/train/d88806d9ece9.png
../data/dr_data_1/train/d88c4843aec3.png
../data/dr_data_1/train/d8cdb7d7283a.png
../data/dr_data_1/train/d8da9de62743.png
../data/dr_data_

../data/dr_data_1/train/e4f12411fd85.png
../data/dr_data_1/train/e50b0174690d.png
../data/dr_data_1/train/e5197d77ec68.png
../data/dr_data_1/train/e529c5757d64.png
../data/dr_data_1/train/e52ed5c29c5e.png
../data/dr_data_1/train/e540d2e35d15.png
../data/dr_data_1/train/e55188915f9d.png
../data/dr_data_1/train/e580676516b0.png
../data/dr_data_1/train/e582e56e7942.png
../data/dr_data_1/train/e594c19e2e1d.png
../data/dr_data_1/train/e599151ca14b.png
../data/dr_data_1/train/e59c5f345bb0.png
../data/dr_data_1/train/e5d56f4f359b.png
../data/dr_data_1/train/e5de79795c1d.png
../data/dr_data_1/train/e5f332efcbc7.png
../data/dr_data_1/train/e5f73f2855c0.png
../data/dr_data_1/train/e60e4edb3ca9.png
../data/dr_data_1/train/e62490b7d0e9.png
../data/dr_data_1/train/e632e38fd2d4.png
../data/dr_data_1/train/e6552b7432b3.png
../data/dr_data_1/train/e65a2ff90494.png
../data/dr_data_1/train/e65f94ad9be3.png
../data/dr_data_1/train/e663c6627a95.png
../data/dr_data_1/train/e66855a5c583.png
../data/dr_data_